In [ ]:
from CONSTANTS import char2idx, idx2char, max_smile_len, min_smile_len, vocab_size, device
import pandas as pd
import torch.optim as optim
import Generator as Generator
import Discriminator as Discriminator
import torch
from train_utilis import  load_checkpoint,pre_train, main_train
import pandas as pd
from pprint import pprint
from test_utilis import evaluate_smiles

In [ ]:
df=pd.read_csv("/kaggle/input/antivirus-generation/train_data.csv")
df = df[
    df['0'].str.len().between(min_smile_len, max_smile_len-2)
]
drugs=df['0'].unique()
generator = Generator(vocab_size).to(device)
discriminator = Discriminator(vocab_size).to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)


g_learning_rate = 2e-4  # or 2e-4
d_learning_rate = 1e-3  # or 2e-4
beta1 = 0.5
beta2 = 0.9  # or 0.999 depending on empirical stability
g_optimizer = torch.optim.Adam(generator.parameters(), lr=g_learning_rate, betas=(beta1, beta2))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=d_learning_rate, betas=(beta1, beta2))

g_scaler = torch.amp.GradScaler('cuda')  
d_scaler = torch.amp.GradScaler('cuda')  

# path="/kaggle/input/gans-checkpoints/other/default/15/checkpoint_epoch_38.pt"
# start_point=load_checkpoint(generator, discriminator, g_optimizer, d_optimizer,g_scaler,d_scaler,path)


### pre training

In [ ]:
pre_train(generator, discriminator,g_optimizer,d_optimizer,g_scaler, d_scaler,drugs, max_len=128)

### main training 

In [ ]:
main_train(generator, discriminator,g_optimizer,d_optimizer,g_scaler,d_scaler, drugs, max_len=128))

### testing generator

In [ ]:

smiles_list=[]
with torch.no_grad():
    generated, _ = generator.sample(char2idx['<SOS>'], 128,10000,temperature=1)
    for i in range(10000):
        smiles = ''.join([idx2char[t] for t in generated[i].tolist() if t !=char2idx['<SOS>'] and t !=char2idx['<EOS>'] and t !=char2idx['<PAD>'] ])
        smiles_list.append(smiles)
metrics = evaluate_smiles(smiles_list, training_set=drugs)
pprint(metrics)